## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim


In [1]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
#from MaaSSim.performance import kpi_pax_pricing
#from MaaSSim.performance import kpi_veh_pricing


import logging
import matplotlib.pyplot as plt


import pandas as pd
import ExMAS

## Delft, Netherlands

In [2]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 50 # reuqests (and passengers)
params.nV = 20 # vehicles


## Parameters for ExMAS

In [3]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)


In [4]:
params.kpi = 1


### Profit Mazimization - Begin 

In [5]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

27-07-23 10:47:20-INFO-Initializing pairwise trip shareability between 50 and 50 trips.
27-07-23 10:47:20-INFO-creating combinations
27-07-23 10:47:20-INFO-2450	 nR*(nR-1)
27-07-23 10:47:20-INFO-Reduction of feasible pairs by 99.55%
27-07-23 10:47:20-INFO-Degree 2 	Completed
27-07-23 10:47:20-INFO-trips to extend at degree 2 : 33
27-07-23 10:47:20-INFO-At degree 2 feasible extensions found out of 2 searched
27-07-23 10:47:20-INFO-Degree 3 	Completed
27-07-23 10:47:20-INFO-trips to extend at degree 3 : 2
27-07-23 10:47:20-INFO-At degree 3 feasible extensions found out of 0 searched
27-07-23 10:47:20-INFO-Degree 4 	Completed
27-07-23 10:47:20-INFO-Max degree reached 4
27-07-23 10:47:20-INFO-Trips still possible to extend at degree 4 : 0


In [6]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

27-07-23 10:47:22-INFO-Matching 50 trips to 85 rides in order to min u_veh
27-07-23 10:47:22-INFO-Problem solution: Optimal. 
Total costs for single trips:         13,823 
reduced by matching to:               12,653


In [7]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],9.319000,503,1,[9.319],"[0, 503]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44860709, 1402734993]","[None, 0, 0]",node time req_id od 0 ...,503,5.03,7.558500,1.511700,6.0468
1,[1],13.748500,743,1,[13.7485],"[54, 743]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1402613322, 44763127]","[None, 1, 1]",node time req_id od 0 ...,743,7.43,11.148000,2.229600,8.9184
2,[2],2.129000,115,1,[2.129],"[148, 115]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1448535905, 44770871]","[None, 2, 2]",node time req_id od 0 ...,115,1.15,1.726500,0.345300,1.3812
3,[3],4.467500,241,1,[4.4675],"[175, 241]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44884118, 44823645]","[None, 3, 3]",node time req_id od 0 NaN...,241,2.41,3.624000,0.724800,2.8992
4,[4],4.120500,222,1,[4.1205],"[212, 222]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44844067, 1393024899]","[None, 4, 4]",node time req_id od 0 ...,222,2.22,3.343500,0.668700,2.6748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,"[42, 43]",8.883000,361,21,"[5.079125, 3.8038749999999997]","[2840.5, 50, 236, 75]","[42, 43]","[43, 42]",2,80,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1448535775, 4474353180, 4228809539, 447...","[None, 42, 43, 43, 42]",node time req_id od 0 ...,361,3.61,6.016500,1.203300,4.8132
81,"[44, 42]",13.205825,561,21,"[8.113050000000001, 5.092775]","[2871.5, 68, 298, 195]","[44, 42]","[42, 44]",2,81,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1668136707, 1448535775, 44791810, 47157...","[None, 44, 42, 42, 44]",node time req_id od 0 ...,561,5.61,8.546625,1.709325,6.8373
82,"[44, 43]",12.285475,542,21,"[8.103950000000001, 4.181525]","[2854.5, 119, 236, 187]","[44, 43]","[43, 44]",2,82,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1668136707, 4474353180, 4228809539, 471...","[None, 44, 43, 43, 44]",node time req_id od 0 ...,542,5.42,7.840125,1.568025,6.2721
83,"[12, 10, 14]",18.618450,693,30,"[4.856175, 6.97205, 6.790225]","[789.0, 83, 151, 84, 214, 161]","[12, 10, 14]","[12, 10, 14]",3,83,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 620833182, 1530303715, 44855130, 448296...","[None, 12, 10, 14, 12, 10, 14]",node time req_id od 0 ...,693,6.93,12.093750,2.418750,9.6750


In [8]:
#sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)

In [23]:
sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate

0.0005
27-07-23 11:23:07-CRITICAL-this is request 1 with [1, 73] available rides.
27-07-23 11:23:07-CRITICAL-ride 1 available [1]
27-07-23 11:23:07-CRITICAL-ride 73 available [1, 0]
27-07-23 11:23:07-CRITICAL-this is request 1 with [1, 73] still available rides.
27-07-23 11:23:07-CRITICAL-vehicle 5 has 2 choices
0.0005
27-07-23 11:23:07-CRITICAL-this is request 2 with [2] available rides.
27-07-23 11:23:07-CRITICAL-ride 2 available [1]
27-07-23 11:23:07-CRITICAL-this is request 2 with [2] still available rides.
27-07-23 11:23:07-CRITICAL-vehicle 20 has 1 choices
0.0005
27-07-23 11:23:07-CRITICAL-this is request 3 with [3, 50] available rides.
27-07-23 11:23:07-CRITICAL-ride 3 available [1]
27-07-23 11:23:07-CRITICAL-ride 50 available [1, 0]
27-07-23 11:23:07-CRITICAL-this is request 3 with [3, 50] still available rides.
27-07-23 11:23:07-CRITICAL-vehicle 6 has 2 choices
0.0005
27-07-23 11:23:07-CRITICAL-this is request 5 with [5] available rides.
27-07-23 11:23:07-CRITICAL-ride 5 avail

In [27]:
sim.res[0].pax_exp

,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,LOSES_PATIENCE,MEETS_DRIVER_AT_PICKUP,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,RECEIVES_OFFER,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,TRAVEL,WAIT,OPERATIONS
pax,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,615.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0
1,15.0,743.0,20.0,30.0,0.0,239.0,54.0,10.0,0,0,0,0,0,0,743.0,239.0,55.0
2,15.0,115.0,20.0,30.0,0.0,89.0,148.0,10.0,0,0,0,0,0,0,115.0,89.0,55.0
3,15.0,241.0,20.0,30.0,0.0,13.0,175.0,10.0,0,0,0,0,0,0,241.0,13.0,55.0
4,0.0,0.0,0.0,0.0,615.0,0.0,212.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0
5,15.0,233.0,20.0,30.0,0.0,44.0,464.0,10.0,0,0,0,0,0,0,233.0,44.0,55.0
6,15.0,106.0,20.0,30.0,0.0,24.0,476.0,10.0,0,0,0,0,0,0,106.0,24.0,55.0
7,0.0,0.0,0.0,0.0,615.0,0.0,574.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0
8,15.0,95.0,20.0,30.0,0.0,44.0,606.0,10.0,0,0,0,0,0,0,95.0,44.0,55.0


In [ ]:
sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[False]

In [26]:
Fulfilled_requests = sim.res[0].pax_exp.LOSES_PATIENCE.value_counts()[False]
unFulfilled_requests = sim.res[0].pax_exp.LOSES_PATIENCE.value_counts()[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')

KeyError: False

In [15]:
sim.vehs[5].rdf 

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,sim_schedule,ttrav,dist,fare,commission,driver_revenue,pickup_dist,trav_dist,operating_cost,profit
2,[2],2.1290,115,1,[2.129],"[148, 115]",[2],[2],1,2,...,node time req_id od 0 ...,115,1.15,1.72650,0.34530,1.3812,409,410.15,0.205075,1.176125
63,"[20, 27]",8.5096,507,20,"[5.087400000000001, 3.4222]","[1378.5, 262, 167, 78]","[20, 27]","[20, 27]",2,63,...,node time req_id od 0 ...,507,5.07,5.42925,1.08585,4.3434,649,654.07,0.327035,4.016365
20,[20],5.1445,278,1,[5.1445],"[1377, 278]",[20],[20],1,20,...,node time req_id od 0 ...,278,2.78,4.17150,0.83430,3.3372,649,651.78,0.325890,3.011310
41,[41],9.4025,508,1,[9.4025],"[2794, 508]",[41],[41],1,41,...,node time req_id od 0 ...,508,5.08,7.62450,1.52490,6.0996,2114,2119.08,1.059540,5.040060


In [16]:
sim.inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],9.319000,503,1,[9.319],"[0, 503]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44860709, 1402734993]","[None, 0, 0]",node time req_id od 0 ...,503,5.03,7.558500,1.511700,6.0468
1,[1],13.748500,743,1,[13.7485],"[54, 743]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1402613322, 44763127]","[None, 1, 1]",node time req_id od 0 ...,743,7.43,11.148000,2.229600,8.9184
2,[2],2.129000,115,1,[2.129],"[148, 115]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1448535905, 44770871]","[None, 2, 2]",node time req_id od 0 ...,115,1.15,1.726500,0.345300,1.3812
3,[3],4.467500,241,1,[4.4675],"[175, 241]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44884118, 44823645]","[None, 3, 3]",node time req_id od 0 NaN...,241,2.41,3.624000,0.724800,2.8992
4,[4],4.120500,222,1,[4.1205],"[212, 222]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44844067, 1393024899]","[None, 4, 4]",node time req_id od 0 ...,222,2.22,3.343500,0.668700,2.6748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,"[42, 43]",8.883000,361,21,"[5.079125, 3.8038749999999997]","[2840.5, 50, 236, 75]","[42, 43]","[43, 42]",2,80,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1448535775, 4474353180, 4228809539, 447...","[None, 42, 43, 43, 42]",node time req_id od 0 ...,361,3.61,6.016500,1.203300,4.8132
81,"[44, 42]",13.205825,561,21,"[8.113050000000001, 5.092775]","[2871.5, 68, 298, 195]","[44, 42]","[42, 44]",2,81,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1668136707, 1448535775, 44791810, 47157...","[None, 44, 42, 42, 44]",node time req_id od 0 ...,561,5.61,8.546625,1.709325,6.8373
82,"[44, 43]",12.285475,542,21,"[8.103950000000001, 4.181525]","[2854.5, 119, 236, 187]","[44, 43]","[43, 44]",2,82,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1668136707, 4474353180, 4228809539, 471...","[None, 44, 43, 43, 44]",node time req_id od 0 ...,542,5.42,7.840125,1.568025,6.2721
83,"[12, 10, 14]",18.618450,693,30,"[4.856175, 6.97205, 6.790225]","[789.0, 83, 151, 84, 214, 161]","[12, 10, 14]","[12, 10, 14]",3,83,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 620833182, 1530303715, 44855130, 448296...","[None, 12, 10, 14, 12, 10, 14]",node time req_id od 0 ...,693,6.93,12.093750,2.418750,9.6750


In [17]:
pd.DataFrame(sim.vehs[4].myrides)

,veh,pos,t,event,paxes
0,4,4.489332e+07,0,STARTS_DAY,[]
1,4,4.489332e+07,0,OPENS_APP,[]
2,4,4.489332e+07,3384,RECEIVES_REQUEST,[]
3,4,4.489332e+07,3399,ACCEPTS_REQUEST,[]
4,4,4.489332e+07,3419,IS_ACCEPTED_BY_TRAVELLER,[None]
5,4,1.391417e+09,3489,ARRIVES_AT_PICKUP,[]
6,4,1.391417e+09,3489,MEETS_TRAVELLER_AT_PICKUP,[]
7,4,1.391417e+09,3519,DEPARTS_FROM_PICKUP,[48]
8,4,4.484488e+07,3653,ARRIVES_AT_DROPOFF,[48]
9,4,4.484488e+07,10799,ENDS_SHIFT,[]


In [18]:
responses = []
avg_kpi = []
idle_time = []

for i in range(1, 4):
    params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
    sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))
    #driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)['Vehicles'] = sim.res[0].veh_exp.index
    #sim.res[0].veh_exp['ds'] = f"{i}"
    
    #responses.append(sim.res[0].veh_exp)
     
   # vehicles = sim.res[0].veh_exp.loc[sim.res[0].veh_exp["nRIDES"] > 0]
    #no_of_veh = len(vehicles)
    
    #avg_kpi.append(sim.res[0].all_kpi/no_of_veh)
    #idle_time.append(vehicles['IDLE'].sum()/no_of_veh)
    
    #_inData.sblts.rides['commission'] = _inData.sblts.rides.apply(lambda row: row.fare*sp.comm_rate, axis=1)
    

0.0005
27-07-23 10:49:05-CRITICAL-this is request 1 with [1, 73] available rides.
27-07-23 10:49:05-CRITICAL-ride 1 available [1]
27-07-23 10:49:05-CRITICAL-ride 73 available [1, 0]
27-07-23 10:49:05-CRITICAL-this is request 1 with [1, 73] still available rides.
27-07-23 10:49:05-CRITICAL-vehicle 8 has 2 choices
0.0005
27-07-23 10:49:05-CRITICAL-this is request 2 with [2] available rides.
27-07-23 10:49:05-CRITICAL-ride 2 available [1]
27-07-23 10:49:05-CRITICAL-this is request 2 with [2] still available rides.
27-07-23 10:49:05-CRITICAL-vehicle 6 has 1 choices
0.0005
27-07-23 10:49:05-CRITICAL-this is request 3 with [3, 50] available rides.
27-07-23 10:49:05-CRITICAL-ride 3 available [1]
27-07-23 10:49:05-CRITICAL-ride 50 available [1, 0]
27-07-23 10:49:05-CRITICAL-this is request 3 with [3, 50] still available rides.
27-07-23 10:49:06-CRITICAL-vehicle 11 has 2 choices
0.0005
27-07-23 10:49:06-CRITICAL-this is request 5 with [5] available rides.
27-07-23 10:49:06-CRITICAL-ride 5 avail

In [19]:
Fulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[False]
unFulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')

AttributeError: 'DataFrame' object has no attribute 'LOST_PATIENCE'